# Initial AI Agent A2C

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
%cd "J:\git\TensorFlowPSX\Py"

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 2000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'discreteAccel' : False,
  'accelAndBrake' : False,
  'discSteer' : False,
  'contAccelOnly' : True,
  'discAccelOnly' : False,
  'modelMode': 4,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 4, # 1 is High Speed Ring x MR2, 2 is 0-400m x MR2, 3 is 0-400 Supra disc, 4 is 0-400 Supra cont, 
}

j:\git\TensorFlowPSX\Py


In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-04 21:31:11,953	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.A2C import A2CConfig

algo = (
    A2CConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=True,
        )
    .training(
        train_batch_size=128,
    )
    .build()
)


#SACConfig.framework("")

2023-10-04 21:31:14,618	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-10-04 21:31:15,251	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignor

(RolloutWorker pid=4376) GT Real Time instantiated
(RolloutWorker pid=4376) GT AI Server instantiated for rtgym
(RolloutWorker pid=4376) starting up on localhost port 9999
(RolloutWorker pid=4376) Waiting for a connection


(RolloutWorker pid=4376) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float64
(RolloutWorker pid=4376)   logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(RolloutWorker pid=4376) 2023-10-04 21:31:21,216	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=4376) 2023-10-04 21:31:21,216	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=4376) 2023-10-04 21:31:21,220	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=4376) 2023-10-04 21:31:21,220	WAR

(RolloutWorker pid=4376) Connection from ('127.0.0.1', 55810)
(RolloutWorker pid=4376) control 6 model mode 4
(RolloutWorker pid=4376) reset triggered
(RolloutWorker pid=4376) reload save for track : 5
(RolloutWorker pid=4376) mode 6 0.0


(RolloutWorker pid=4376) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=4376)   if not isinstance(terminated, (bool, np.bool8)):
(RolloutWorker pid=4376) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: int64
(RolloutWorker pid=4376)   logger.warn(
(RolloutWorker pid=4376) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
(RolloutWorker pid=4376)   logger.warn(f"{pre} is not within the observation space.")


(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.9887199997901917
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.9440001249313354
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.5395752787590027
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.8299592733383179
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.7841014862060547
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.735425591468811
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.44032514095306396
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.588037371635437
(Roll

2023-10-04 21:31:23,290	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_action_dist.TorchDiagGaussian` has been deprecated. Use `ray.rllib.models.torch.torch_distributions.TorchDiagGaussian` instead. This will raise an error in the future!
2023-10-04 21:31:23,290	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_action_dist.TorchDistributionWrapper` has been deprecated. Use `ray.rllib.models.torch.torch_distributions.` instead. This will raise an error in the future!


(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.6821128726005554
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.5953394770622253


In [6]:
N = 1000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.5330452919006348
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.8049622774124146
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.930709958076477
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.1510026752948761
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.5543584227561951
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.6331260204315186
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.39535412192344666
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.9244672060012817
(Rol

(RolloutWorker pid=4376) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.135384299636371
(RolloutWorker pid=4376)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.6099129319190979
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False

2023-10-04 21:33:07,104	WARNING tensorboardx.py:88 -- You are trying to log an invalid value (ray/tune/hist_stats/episode_reward=[0.0]) via TBXLogger!


(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
Loop:  16
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(Rol

...

(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0
(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 0.0


(RolloutWorker pid=4376) Exception in thread Thread-2:
(RolloutWorker pid=4376) Traceback (most recent call last):
(RolloutWorker pid=4376)   File "python\ray\_raylet.pyx", line 1498, in ray._raylet.execute_task
(RolloutWorker pid=4376)   File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
(RolloutWorker pid=4376)   File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=4376)   File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
(RolloutWorker pid=4376)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=4376)   File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
(RolloutWorker pid=4376)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=4376)   File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rlli

(RolloutWorker pid=4376) 0 0.0 False
(RolloutWorker pid=4376) mode 6 1.0
(RolloutWorker pid=4376) 0 0.0 False


RayTaskError(ValueError): [36mray::RolloutWorker.apply()[39m (pid=4376, ip=127.0.0.1, actor_id=3f00e83d477138a1aa277a9401000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000150ACD4C730>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\utils\actor_manager.py", lin

2023-10-04 21:44:47,517	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RolloutWorker.apply() (pid=4376, ip=127.0.0.1, actor_id=3f00e83d477138a1aa277a9401000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000150ACD4C730>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\en